# Naive Bayes for Spam Classification

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [6]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('d:/Python Projects/Email/spam', 'spam'))
data = data.append(dataFrameFromDirectory('d://Python Projects/Email/ham', 'ham'))


Let's have a look at that DataFrame:

In [8]:
data.tail()

,class,message
d://Python Projects/Email/ham\02496.aae0c81581895acfe65323f344340856,ham,Man killed 'trying to surf' on Tube train \n\n...
d://Python Projects/Email/ham\02497.60497db0a06c2132ec2374b2898084d3,ham,"Hi Gianni,\n\n\n\nA very good resource for thi..."
d://Python Projects/Email/ham\02498.09835f512f156da210efb99fcc523e21,ham,Gianni Ponzi wrote:\n\n> I have a prob when tr...
d://Python Projects/Email/ham\02499.b4af165650f138b10f9941f6cc5bce3c,ham,Neale Pickett <neale@woozle.org> writes:\n\n\n...
d://Python Projects/Email/ham\02500.05b3496ce7bca306bed0805425ec8621,ham,"\n\nHi,\n\n\n\nI think you need to give us a l..."


Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [10]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Let's try it out:

In [11]:
examples = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'], 
      dtype='|S4')